<a href="https://colab.research.google.com/github/kshitijmamgain/Higgs/blob/master/LGBM_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

     |████████████████████████████████| 163kB 440kB/s 
     |████████████████████████████████| 1.1MB 41.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 112kB 50.4MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=94474d425bc540a014e222a94b8cbb42b8301e4da0bd8e0d9eecab0e11d90838
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-1.3.0-cp36-none-any.whl size=221121 sha256=3433fe26bcf5e36ec92efd7afc8303a0ffbdccf229be230bf5e4d7210cead603
  Stored in directory: /root/.cache/pip/wheels/62/f

In [0]:
# import dependent libraries

'''
#reference links
Hyperopt and Randomsearch with early stopping https://github.com/WillKoehrsen/hyperparameter-optimization 
Optuna tuner https://github.com/optuna/optuna/blob/master/examples/lightgbm_tuner_simple.py
'''

import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import RandomizedSearchCV

import csv
from timeit import default_timer as timer
from hyperopt import STATUS_OK, hp, tpe, Trials, fmin
from hyperopt.pyll.stochastic import sample
import random

import optuna.integration.lightgbm as lgbo
import optuna


In [0]:
# starting with storing the data as data frame
df = pd.read_csv("drive/My Drive/Colab Notebooks/train_test_files_sample.csv")
df.drop( columns='Unnamed: 0', inplace =True)

# making a smaller df for quick testing
df_s, _ = train_test_split(df, random_state = 30, train_size = 0.01)

train_X = df_s.drop(columns = '0')
train_y = df_s['0']

In [0]:
# drop last columns

def col_keep(df):
  return df.drop(columns = list(map(str,range(22,29))), inplace = True) # removing 7 last columns


In [0]:
MAX_EVALS = 1000
N_FOLDS = 3
NUM_BOOST_ROUNDS = 10000
EARLY_STOPPING_ROUNDS = 100
SEED = 47

# Hyperopt

In [6]:
def objective(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization"""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Retrieve the subsample if present otherwise set to 1.0
    subsample = params['boosting_type'].get('subsample', 1.0)
    
    # Extract the boosting type
    params['boosting_type'] = params['boosting_type']['boosting_type']
    params['subsample'] = subsample
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_data_in_leaf', 
                           'max_bin', 'bagging_freq']:
        params[parameter_name] = int(params[parameter_name])
    
    start = timer()
    
    # Perform n_folds cross validation
    cv_results = lgb.cv(params, train_set, num_boost_round = 10000, nfold = n_folds, 
                        early_stopping_rounds = 100, metrics = ['auc','binary','xentropy'], seed = 50,
                        )
    
    run_time = timer() - start
    
    # Extract the best score
    best_score = np.max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)

    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, n_estimators, run_time])
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'iteration': ITERATION,
            'estimators': n_estimators, 
            'train_time': run_time, 'status': STATUS_OK}


# Hyperopt Space
space = {              
            'num_leaves': hp.quniform('num_leaves', 16, 196, 4),

            'max_bin' : hp.quniform('max_bin', 253, 254, 1), #if using CPU just set this to 254

            'lambda_l1': hp.uniform('lambda_l1', 0.0, 1.0),
            
            'lambda_l2': hp.uniform("lambda_l2", 0.0, 1.0),
            
            'min_data_in_leaf' : hp.quniform('min_data_in_leaf', 20, 500, 10),

            'class_weight': hp.choice('class_weight', [None, 'balanced']),
            
            'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                                         #{'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                                                         {'boosting_type': 'goss', 'subsample': 1.0}]),

            'learning_rate' : hp.loguniform('learning_rate', np.log(0.05), np.log(0.25)),

            'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),

            'feature_fraction': hp.uniform('feature_fraction', 0.4, 1.0),
                     
            'bagging_freq': hp.uniform('bagging_freq', 1, 7),
                     
            'verbosity' : 0,
            
            }

# optimization algorithm
tpe_algorithm = tpe.suggest

# Keep track of results
bayes_trials = Trials()

# File to save first results
out_file = 'gbm_trials.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()

# Global variable
global  ITERATION

ITERATION = 0

train_set = lgb.Dataset(data=train_X, label = train_y)

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, rstate = np.random.RandomState(50))

100%|██████████| 100/100 [03:46<00:00,  2.26s/it, best loss: 0.21499099529680832]


# Random Search

In [27]:
def random_objective(params, iteration, n_folds = N_FOLDS):
    """Random search objective function. Takes in hyperparameters
       and returns a list of results to be saved."""

    start = timer()
    # Subsampling (only applicable with 'goss')
    subsample_dist = list(np.linspace(0.5, 1, 100))

    if params['boosting_type'] == 'goss':
        # Cannot subsample with goss
        params['subsample'] = 1.0
    else:
        # Subsample supported for gdbt and dart
        params['subsample'] = random.sample(subsample_dist, 1)[0]    

    # Perform n_folds cross validation
    cv_results = lgb.cv(params, train_set, num_boost_round = 1000, nfold = n_folds, 
                        early_stopping_rounds = 100, metrics = ['auc','binary','xentropy'], seed = 50)
    end = timer()
    best_score = np.max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)
    
    # Return list of results
    return [loss, params, iteration, n_estimators, end - start]

# Hyperparameter grid
param_grid = {
                'num_leaves': list(range(16, 196, 4)),
              
                'max_bin': [254],

                'lambda_l1': list(np.linspace(0, 1)),

                'lambda_l2': list(np.linspace(0, 1)),

                'min_data_in_leaf' : list(range(20, 500, 10)),

                'class_weight': [None, 'balanced'],

                'boosting_type': ['gbdt', 'goss'],
              
                'learning_rate' : list(np.logspace(np.log(0.05), np.log(0.2), base = np.exp(1), num = 1000)),

                'feature_fraction': list(np.linspace(0.4, 1.0)),

                'bagging_freq': list(range(1,7)),

                'verbosity' : [0]
                }

random.seed(50)

# Dataframe to hold cv results
random_results = pd.DataFrame(columns = ['loss', 'params', 'iteration', 'estimators', 'time'],
                       index = list(range(MAX_EVALS)))

# Iterate through the specified number of evaluations
for i in range(15):
    
    # Randomly sample parameters for gbm
    params = {key: random.sample(value, 1)[0] for key, value in param_grid.items()}
    
    print(params)  
        
    results_list = random_objective(params, i)
    
    # Add results to next row in dataframe
    random_results.loc[i, :] = results_list

{'num_leaves': 140, 'max_bin': 254, 'lambda_l1': 0.4693877551020408, 'lambda_l2': 0.8163265306122448, 'min_data_in_leaf': 170, 'class_weight': 'balanced', 'boosting_type': 'goss', 'learning_rate': 0.05641591746163987, 'feature_fraction': 0.8163265306122449, 'bagging_freq': 3, 'verbosity': 0}
{'num_leaves': 188, 'max_bin': 254, 'lambda_l1': 0.18367346938775508, 'lambda_l2': 0.44897959183673464, 'min_data_in_leaf': 80, 'class_weight': 'balanced', 'boosting_type': 'goss', 'learning_rate': 0.06851322752809227, 'feature_fraction': 0.5469387755102041, 'bagging_freq': 1, 'verbosity': 0}
{'num_leaves': 168, 'max_bin': 254, 'lambda_l1': 0.8163265306122448, 'lambda_l2': 0.1020408163265306, 'min_data_in_leaf': 60, 'class_weight': 'balanced', 'boosting_type': 'goss', 'learning_rate': 0.05034812683758164, 'feature_fraction': 0.5714285714285714, 'bagging_freq': 3, 'verbosity': 0}
{'num_leaves': 124, 'max_bin': 254, 'lambda_l1': 0.24489795918367346, 'lambda_l2': 0.5714285714285714, 'min_data_in_leaf'

# Optuna

In [7]:
def objective(trial):
    
    dtrain = lgbo.Dataset(train_X, label=train_y)

    global ITERATION_O

    ITERATION_O += 1

    param ={              
            'num_leaves': trial.suggest_int('num_leaves', 16, 196, 4),

            'max_bin' : trial.suggest_discrete_uniform('max_bin', 63, 255,64), #if using CPU just set this to 254

            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
            
            'lambda_l2': trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
            
            'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 20, 500),

            'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
            
            'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'goss']), # removed 'dart'

            'learning_rate' : trial.suggest_loguniform('learning_rate', 0.05, 0.25),

            'feature_fraction': trial.suggest_uniform("feature_fraction", 0.4, 1.0),
                
            'bagging_freq': trial.suggest_int("bagging_freq", 1, 7),
                     
            'verbosity' : 0

        }
    
    start = timer()

        # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'min_data_in_leaf', 
                           'max_bin', 'bagging_freq']:
        param[parameter_name] = int(param[parameter_name])

    # Perform n_folds cross validation
    if param['boosting_type'] == 'goss':
      param['subsample'] = 1
    else:
      param['subsample'] = trial.suggest_uniform('subsample', 0.5, 1)
    
    cv_results = lgb.cv(param, dtrain, num_boost_round = 10000, nfold = 3, 
                        early_stopping_rounds = 100, metrics = ['auc','binary','xentropy'], 
                        seed = 50,
                        )
    
    run_time = timer() - start
    
    # Extract the best score
    best_score = np.max(cv_results['auc-mean'])

    loss = 1 - best_score

    # Boosting rounds that returned the highest cv score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)

    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, param, ITERATION_O, n_estimators, run_time])

    
    return loss

# Global variable
global  ITERATION_O

ITERATION_O = 0

# File to save first results
out_file = 'gbm_optuna.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()

study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=100)

[I 2020-04-06 19:52:41,739] Finished trial#0 with value: 0.23937060412755595 with parameters: {'num_leaves': 108, 'max_bin': 191.0, 'lambda_l1': 0.09268139426113875, 'lambda_l2': 6.36355407290065, 'min_data_in_leaf': 105, 'class_weight': 'balanced', 'boosting_type': 'goss', 'learning_rate': 0.19774658176662815, 'feature_fraction': 0.709128398650851, 'bagging_freq': 6}. Best is trial#0 with value: 0.23937060412755595.
[I 2020-04-06 19:52:42,895] Finished trial#1 with value: 0.24001519622309075 with parameters: {'num_leaves': 140, 'max_bin': 255.0, 'lambda_l1': 7.89492562093299e-05, 'lambda_l2': 0.007971569858429745, 'min_data_in_leaf': 142, 'class_weight': None, 'boosting_type': 'goss', 'learning_rate': 0.08620222449969922, 'feature_fraction': 0.9416601400713059, 'bagging_freq': 1}. Best is trial#0 with value: 0.23937060412755595.
[I 2020-04-06 19:52:44,499] Finished trial#2 with value: 0.2285245843880035 with parameters: {'num_leaves': 192, 'max_bin': 63.0, 'lambda_l1': 0.0046712462857

In [23]:
study.trials

[FrozenTrial(number=0, value=0.23937060412755595, datetime_start=datetime.datetime(2020, 4, 6, 19, 52, 40, 685825), datetime_complete=datetime.datetime(2020, 4, 6, 19, 52, 41, 739079), params={'num_leaves': 108, 'max_bin': 191.0, 'lambda_l1': 0.09268139426113875, 'lambda_l2': 6.36355407290065, 'min_data_in_leaf': 105, 'class_weight': 'balanced', 'boosting_type': 'goss', 'learning_rate': 0.19774658176662815, 'feature_fraction': 0.709128398650851, 'bagging_freq': 6}, distributions={'num_leaves': IntUniformDistribution(high=196, low=16, step=4), 'max_bin': DiscreteUniformDistribution(high=255, low=63, q=64), 'lambda_l1': LogUniformDistribution(high=10.0, low=1e-08), 'lambda_l2': LogUniformDistribution(high=10.0, low=1e-08), 'min_data_in_leaf': IntUniformDistribution(high=500, low=20, step=1), 'class_weight': CategoricalDistribution(choices=(None, 'balanced')), 'boosting_type': CategoricalDistribution(choices=('gbdt', 'goss')), 'learning_rate': LogUniformDistribution(high=0.25, low=0.05), 

In [28]:
pd.DataFrame(bayes_trials.results)

,loss,params,iteration,estimators,train_time,status
0,0.216957,"{'bagging_freq': 2, 'boosting_type': 'gbdt', '...",1,108,2.337818,ok
1,0.244583,"{'bagging_freq': 3, 'boosting_type': 'goss', '...",2,18,0.807286,ok
2,0.219852,"{'bagging_freq': 5, 'boosting_type': 'gbdt', '...",3,185,2.213276,ok
3,0.222999,"{'bagging_freq': 5, 'boosting_type': 'gbdt', '...",4,47,1.385302,ok
4,0.225626,"{'bagging_freq': 5, 'boosting_type': 'gbdt', '...",5,112,1.826035,ok
...,...,...,...,...,...,...
95,0.224094,"{'bagging_freq': 2, 'boosting_type': 'gbdt', '...",96,31,2.798928,ok
96,0.216322,"{'bagging_freq': 2, 'boosting_type': 'gbdt', '...",97,81,2.659975,ok
97,0.218820,"{'bagging_freq': 3, 'boosting_type': 'gbdt', '...",98,116,3.143319,ok
98,0.240561,"{'bagging_freq': 1, 'boosting_type': 'goss', '...",99,19,0.994468,ok


# LGBM Class

In [0]:
'''
reference https://towardsdatascience.com/an-example-of-hyperparameter-optimization-on-xgboost-lightgbm-and-catboost-using-hyperopt-12bc41a271e

'''
from sklearn.metrics import mean_squared_error
# LightGBM parameters
lgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
}
lgb_fit_params = {
    'eval_metric': 'l2',
    'early_stopping_rounds': 10,
    'verbose': False
}
lgb_para = dict()
lgb_para['reg_params'] = lgb_reg_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))


class LightGBM_Class():
  
  def __init__(self, x_train, x_test, y_train, y_test):
    
    self.x_train = x_train
    self.x_test  = x_test
    self.y_train = y_train
    self.y_test  = y_test

  
  def process(self, fn_name, space, trials, algo, max_evals):
    fn = getattr(self, fn_name)
    try:
        result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
    except Exception as e:
        return {'status': STATUS_FAIL, 'exception': str(e)}
    return result, trials
  
  def lgb_reg(self, para):
    reg = lgb.LGBMClassifier(**para['reg_params'])
    return self.train_reg(reg, para)

  def train_reg(self, reg, para):
    reg.fit(self.x_train, self.y_train,
            eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
            **para['fit_params'])
    pred = reg.predict(self.x_test)
    loss = para['loss_func'](self.y_test, pred)
    return {'loss': loss, 'status': STATUS_OK}
 



In [0]:
x_train, x_test, y_train, y_test = train_test_split(train_X, train_y, random_state = 30, train_size = 0.7)
obj = LightGBM_Class(x_train, x_test, y_train, y_test)

In [36]:
lgb_opt = obj.process(fn_name='lgb_reg', space=lgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [00:39<00:00,  2.53it/s, best loss: 0.5278652300041049]


In [43]:
lgb_opt[1].results


[{'loss': 0.5341406668505004, 'status': 'ok'},
 {'loss': 0.5456035879196046, 'status': 'ok'},
 {'loss': 0.5394615131911961, 'status': 'ok'},
 {'loss': 0.5397555778154831, 'status': 'ok'},
 {'loss': 0.5432719379865095, 'status': 'ok'},
 {'loss': 0.5359201520691709, 'status': 'ok'},
 {'loss': 0.5365120022242049, 'status': 'ok'},
 {'loss': 0.5409302376984393, 'status': 'ok'},
 {'loss': 0.5344376591819557, 'status': 'ok'},
 {'loss': 0.5359201520691709, 'status': 'ok'},
 {'loss': 0.547055822556255, 'status': 'ok'},
 {'loss': 0.5421023522639004, 'status': 'ok'},
 {'loss': 0.5359201520691709, 'status': 'ok'},
 {'loss': 0.548793431450347, 'status': 'ok'},
 {'loss': 0.5400494823174234, 'status': 'ok'},
 {'loss': 0.5499487873217019, 'status': 'ok'},
 {'loss': 0.5429797777405959, 'status': 'ok'},
 {'loss': 0.5400494823174234, 'status': 'ok'},
 {'loss': 0.5412235043167296, 'status': 'ok'},
 {'loss': 0.5429797777405959, 'status': 'ok'},
 {'loss': 0.5344376591819557, 'status': 'ok'},
 {'loss': 0.529